In [41]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer,make_column_transformer,make_column_selector

In [ ]:
# ohe = OneHotEncoder(sparse_output=False, drop='first').set_output(transform='pandas')
# scaler_mm = MinMaxScaler()
# scaler_std = StandardScaler()
# ct = make_column_transformer(('passthrough', make_column_selector(dtype_exclude=object)  ),
#                              (ohe, make_column_selector(dtype_include=object) ),
#                             verbose_feature_names_out=False).set_output(transform='pandas')

In [3]:
df = pd.read_csv('covid_toy.csv')

In [5]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [7]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [11]:
X = df.drop('has_covid',axis=1)
y = df['has_covid']
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=24,test_size=0.3)

In [13]:
X_train

,age,gender,fever,cough,city
49,44,Male,104.0,Mild,Mumbai
9,64,Female,101.0,Mild,Delhi
69,73,Female,103.0,Mild,Delhi
23,80,Female,98.0,Mild,Delhi
74,34,Female,104.0,Strong,Delhi
...,...,...,...,...,...
17,40,Female,98.0,Strong,Delhi
87,47,Male,101.0,Strong,Bangalore
64,42,Male,104.0,Mild,Mumbai
3,31,Female,98.0,Mild,Kolkata


if we don't use column transformer we have to do it this way

In [16]:
# adding simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])
                                 
X_train_fever.shape


(70, 1)

In [18]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(70, 1)

In [22]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(70, 4)

In [24]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(70, 1)

In [77]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(70, 7)

with using column transformer

In [80]:
# transformer = ColumnTransformer(transformers=[
#     ('tnf1',SimpleImputer(),['fever']),
#     ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
#     ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
# ],remainder='passthrough')

In [82]:
ct = make_column_transformer(('passthrough', make_column_selector(dtype_exclude=object)  ),
                             (ohe, make_column_selector(dtype_include=object) ),
                            verbose_feature_names_out=False).set_output(transform='pandas')

In [86]:
ct.fit_transform(X_train).shape

(70, 7)

In [88]:
ct.transform(X_test)

,age,fever,gender_Male,cough_Strong,city_Delhi,city_Kolkata,city_Mumbai
26,19,100.0,0.0,0.0,0.0,1.0,0.0
31,83,103.0,1.0,0.0,0.0,1.0,0.0
37,55,100.0,1.0,0.0,0.0,1.0,0.0
86,25,104.0,1.0,0.0,0.0,0.0,0.0
76,80,100.0,1.0,0.0,0.0,0.0,0.0
14,51,104.0,1.0,0.0,0.0,0.0,0.0
88,5,100.0,0.0,0.0,0.0,1.0,0.0
48,66,99.0,1.0,1.0,0.0,0.0,0.0
71,75,104.0,0.0,1.0,1.0,0.0,0.0
67,65,99.0,1.0,0.0,0.0,0.0,0.0
